In [ ]:
# Neural network on copenhagen
# cx = np.concatenate((cX,lX), axis=0)
# cY = np.concatenate((cy,ly), axis=0)
cx = np.asarray(cX).astype('float32')
cY = np.asarray(cy).astype('float32')



cX_train, cX_testval, cy_train, cy_testval = train_test_split(cx, cY, test_size=0.20, random_state=4)
cX_test, cX_val, cy_test, cy_val = train_test_split(cX_testval, cy_testval, test_size=0.5, random_state=4)

print(cX_train.shape, cX_test.shape, cX_val.shape)

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#normalize our data
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler()
cx_train_sc = X_scaler.fit_transform(cX_train)
cx_val_sc = X_scaler.transform(cX_val)
cx_test_sc = X_scaler.transform(cX_test)

cx_train_sc.shape

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adagrad
#adagrad = 0.02, sgd = 
lr = 0.06
p = 0.2
model = Sequential()
#found 16 to be optimal
model.add(Dense(units=16, input_dim=29, kernel_initializer='normal', activation='relu'))
# p = 0.2 optimal
model.add(Dropout(p))
model.add(Dense(units=64, kernel_initializer='normal', activation='relu'))
# model.add(Dropout(p))
# model.add(Dense(units=16, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, activation='linear', kernel_initializer='normal'))
print(model.summary())
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_msle'
                        , patience = 3
                        , verbose = 1
                        , mode = 'auto'
                        , min_delta = 0.001)

num_epochs = 100

#larger --> increase stockholm score
#smaller --> increase copen score
# too large / too small, predictions converge to same value
b_size = 32
#optimizers: adam, sgd, 
model.compile(loss="msle", optimizer=SGD(learning_rate=lr), metrics = ['msle'])

history = model.fit(cx_train_sc, cy_train,
                   batch_size = b_size,
                   epochs = num_epochs,
                   verbose = 1,
                   validation_data=(cx_val_sc, cy_val),
                   callbacks = [callback])

# def getAccuracy(x,y):
#     predictions = model.predict(x)
#     #APE (absolute percent error)
#     APE = 100*(abs(y-predictions)/y)
#     accuracy = 100-np.mean(APE)
#     return accuracy


# accuracy = getAccuracy(cx_test_sc, cy_test)
# print('Accuracy of model on test set: ', accuracy)

# sx, sY = np.asarray(sX).astype('float32'), np.asarray(sy).astype('float32')
# acc_stock = getAccuracy(sx,sY)
# print('Model evaluated on Stockholm: ', acc_stock)

import matplotlib.pyplot as plt
plt.plot(history.history['msle'])
plt.plot(history.history['val_msle'])
plt.ylabel('msle')
plt.xlabel('epoch')
plt.title('Neural Network')
plt.legend(['train', 'validation'])
plt.show()

In [ ]:
c_pred = model.predict(cx_test_sc)
print("Copenhagen test set")
plotPredVsReal(cy_test, c_pred, 750)
print("Test r2:", round(r2_score(cy_test, c_pred),4))

In [ ]:
sx, sY = np.asarray(sX).astype('float32'), np.asarray(sy).astype('float32')
sx_sc = X_scaler.transform(sx)
s_pred = model.predict(sx_sc)
print("Stockholm evaluated on copenhagen test set")
plotPredVsReal(sY, s_pred, 750)
print("Test r2:", round(r2_score(sY, s_pred),4))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['msle'])
plt.plot(history.history['val_msle'])
plt.ylabel('msle')
plt.xlabel('epoch')
plt.title('Neural Network')
plt.legend(['train', 'validation'])
plt.show()